In [1]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, VBox, Label, FloatText, Layout

In [2]:
sig_present = widgets.FloatSlider(
    value=7,
    min=0,
    max=15,
    step=0.5,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout={'width':'200px'}
)

sig_absent = widgets.FloatSlider(
    value=4,
    min=0,
    max=15,
    step=0.5,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout={'width':'200px'}
)

std = widgets.FloatSlider(
    value=2,
    min=0.5,
    max=5,
    step=0.5,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout={'width':'200px'}
)

crit = widgets.FloatSlider(
    value=5,
    min=0,
    max=15,
    step=0.5,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout={'width':'200px'}
)

In [3]:
sig_present_label = Label('Signal Present', layout={'width':'200px'})
sig_absent_label = Label('Signal Absent', layout={'width':'200px'})
std_label = Label('Standard Deviation', layout={'width':'200px'})
crit_label = Label('Criterion', layout={'width':'200px'})

In [4]:
hit_rate = widgets.FloatText(
    value=0.0,
    disabled=True,
    color='black',
    layout={'width': '100px'},
)

miss_rate = widgets.FloatText(
    value=0.0,
    disabled=True,
    layout={'width': '100px'},
)

fp_rate = widgets.FloatText(
    value=0.0,
    disabled=True,
    layout={'width': '100px'},
)

cr_rate = widgets.FloatText(
    value=0.0,
    disabled=True,
    layout={'width': '100px', 'justify-content': 'flex-end'},
)

dprime = widgets.FloatText(
    value=0.0,
    disabled=True,
    layout={'width': '100px'},
)

In [5]:
hit_rate_label = Label('Hits', layout={'width': '200px'})
miss_rate_label = Label('Misses', layout={'width': '200px'})
fp_rate_label = Label('False Positives', layout={'width': '200px'})
cr_rate_label = Label('Correct Rejections', layout={'width': '200px'})
dp_label = Label('d\'')

In [6]:
hit_rate_label.add_class('hrl')
fp_rate_label.add_class('fprl');

In [7]:
hit_rate.add_class('black-disabled')
miss_rate.add_class('black-disabled')
fp_rate.add_class('black-disabled')
cr_rate.add_class('black-disabled')
dprime.add_class('black-disabled');

In [8]:
rates_layout = widgets.Layout(width='100%',
                              grid_template_rows='auto auto',
                              grid_template_columns='20% 30% 20% 30%')

rate_widget = widgets.GridBox(children = [hit_rate_label, hit_rate,
                              fp_rate_label, fp_rate,
                              miss_rate_label, miss_rate,
                              cr_rate_label, cr_rate],
                              layout=rates_layout)

slider_layout = widgets.Layout(width='100%',
                              grid_template_rows='auto auto',
                              grid_template_columns='20% 30% 20% 30%')

slider_widget = widgets.GridBox(children=[sig_absent_label, sig_absent,
                                          sig_present_label, sig_present,
                                          std_label, std,
                                          crit_label, crit],
                               layout=slider_layout)

In [9]:
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interactive_output
import numpy as np
import matplotlib.pyplot as plt

In [10]:
import matplotlib as mpl
from scipy.stats import norm

mpl.rcParams.update({'font.size': 22})

def gaussian(x, mu, sig):
    unnormed = np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    scaling = np.sqrt(2*np.pi)*sig
    return unnormed / scaling  

def two_curve_slider(mean_signal_present, mean_signal_absent, standard_deviation, criterion):    
    fig, ax = plt.subplots(figsize=(18,5))

    minval = 0
    maxval = 15
    
    xs = np.linspace(minval, maxval, 1000)
    curve_1 = gaussian(xs, mean_signal_present, standard_deviation)
    curve_2 = gaussian(xs, mean_signal_absent, standard_deviation)
    
    idx = np.argmin(np.abs(xs - criterion))
    
    
    truncated_curve_1 = curve_1[idx:]
    truncated_curve_2 = curve_2[idx:]
    
    truncated_xs = np.linspace(criterion, maxval, truncated_curve_1.shape[0])
    
    plt.fill_between(truncated_xs, truncated_curve_2, color='red', alpha=0.4);
    plt.fill_between(truncated_xs, truncated_curve_1, color='green', alpha=0.4);
    
    mr = norm.cdf(criterion, loc=mean_signal_present, scale=standard_deviation)
    cr = norm.cdf(criterion, loc=mean_signal_absent, scale=standard_deviation)
    dp = (mean_signal_present - mean_signal_absent)/float(standard_deviation)
    
    hit_rate.value = '%.2f' % (1-mr)
    miss_rate.value = '%.2f' % mr
    cr_rate.value = '%.2f' % cr
    fp_rate.value = '%.2f' % (1-cr)
    dprime.value = '%.2f' % dp

    plt.axvline(criterion, c='blue', linewidth=3, linestyle='dashed');
    plt.axvline(mean_signal_present, c='green', alpha=0.2)
    plt.axvline(mean_signal_absent, c='red', alpha=0.2)
    plt.plot(xs, curve_1, color='k', linewidth=4);
    plt.plot(xs, curve_2, color='k', linewidth=4);
    plt.xlim((minval, maxval))
    plt.xticks(np.linspace(minval, maxval, 5), np.linspace(minval, maxval, 5))

    ylims = plt.ylim()
    plt.ylim([0, ylims[1]])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.ylabel(r'$P(x)$')
    plt.xlabel(r'$x$')
    plt.title('Normal Distributions')

In [11]:
## ROC Widget
import matplotlib.patches as patches
def plot_ROC(mean2, mean1, sigma, crit):
    fig, ax = plt.subplots(figsize=(8,8))
    
    crits = np.linspace(mean1 - mean1 * 3, mean2 + mean2 * 3, 100)
    hits = 1 - norm.cdf(crits,
                       loc=mean2,
                       scale=sigma)
    fps = 1 - norm.cdf(crits,
                       loc=mean1,
                       scale=sigma)
    
    plt.plot(fps, hits, color='k', linewidth=4)
    plt.plot(hits,hits, color='k', alpha=0.2)
    
    nearest = np.argmin(np.abs(crit-crits))
    plt.plot(fps[nearest], hits[nearest], 'o', markersize=16, color='blue')
    plt.xticks(np.linspace(0,1,5))
    plt.yticks(np.linspace(0,1,5))
    
    
    plt.axvline(fps[nearest], linestyle='dashed', alpha=0.5, color='red')
    plt.axhline(hits[nearest], linestyle='dashed', alpha=0.5, color='green')
    
    red_rect = patches.Rectangle((0,-0.1),fps[nearest],0.07,facecolor='r')
    green_rect = patches.Rectangle((-0.1,0),0.07,hits[nearest],facecolor='g')
    ax.add_patch(red_rect)
    ax.add_patch(green_rect)
    plt.title('ROC Curve')
    plt.xlabel('False Positives')
    plt.ylabel('Hits')
    
roc_widget = interactive_output(plot_ROC, 
         {'mean2':sig_present,
          'mean1':sig_absent,
          'sigma':std,
          'crit':crit})


In [12]:
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='50%')

In [13]:
plot = interactive_output(two_curve_slider, 
         {'mean_signal_present':sig_present,
          'mean_signal_absent':sig_absent,
          'standard_deviation':std,
          'criterion':crit})

sliders_and_plot = VBox([plot, slider_widget, rate_widget])

roc_and_dprime = VBox([roc_widget, HBox([dp_label, dprime])], layout=box_layout)

plot_and_roc = HBox([sliders_and_plot, roc_and_dprime])
display(plot_and_roc)

In [14]:
%%html
<style>
.hrl {
    font-weight: bold;
    color: green;
}
.fprl {
    font-weight: bold;
    color: red;
}
.black-disabled input[type="number"]:disabled {
    opacity: 1;
    font-weight: bold;
}
</style>

In [15]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')